In [ ]:
from AlgorithmImports import *

class ReflexivityAlgorithm(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2000, 1, 1)
        self.SetCash(100000)

        # Equities & Bonds
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        self.tbill = self.AddEquity("SHY", Resolution.Daily).Symbol
        self.long_bonds = self.AddEquity("TLT", Resolution.Daily).Symbol
        self.corp_bonds = self.AddEquity("LQD", Resolution.Daily).Symbol

        # Commodities
        self.gold = self.AddEquity("GLD", Resolution.Daily).Symbol
        self.oil = self.AddEquity("USO", Resolution.Daily).Symbol

        # **FRED Economic Data**
        self.gdp_data = self.AddData(Fred, "GDP", Resolution.Daily).Symbol
        self.unemployment_data = self.AddData(Fred, "UNRATE", Resolution.Daily).Symbol
        self.yield_10y = self.AddData(Fred, "DGS10", Resolution.Daily).Symbol
        self.yield_2y = self.AddData(Fred, "DGS2", Resolution.Daily).Symbol
        self.credit_spread = self.AddData(Fred, "BAMLH0A0HYM2", Resolution.Daily).Symbol
        self.confidence_data = self.AddData(Fred, "UMCSENT", Resolution.Daily).Symbol

        # Track recent macro values (store last 3 data points)
        self.gdp_values = []
        self.unemployment_values = []
        self.credit_spread_values = []
        self.confidence_values = []
        
        # Track highest SPY and Bond prices for stop-loss
        self.spy_trailing_high = 0  
        self.tlt_trailing_high = 0  
        self.lqd_trailing_high = 0  

        # Portfolio-wide max drawdown tracking
        self.portfolio_peak = self.Portfolio.TotalPortfolioValue  

        # ATR Stop-Loss for SPY
        self.atr = self.ATR(self.spy, 14, Resolution.Daily)  

        # Moving Average for Bonds (50-Day)
        self.tlt_sma = self.SMA(self.long_bonds, 50, Resolution.Daily)

        # Warm-up to ensure historical data is available
        self.SetWarmUp(TimeSpan.FromDays(200))  

        # Monthly Rebalancing
        self.Schedule.On(self.DateRules.MonthStart(), self.TimeRules.At(9, 30), self.Rebalance)

    def OnData(self, data):
        # GDP Data
        if self.gdp_data in data and data[self.gdp_data] is not None:
            self.gdp_values.append(float(data[self.gdp_data].Value))
            if len(self.gdp_values) > 3:
                self.gdp_values.pop(0)

        # Unemployment Rate
        if self.unemployment_data in data and data[self.unemployment_data] is not None:
            self.unemployment_values.append(float(data[self.unemployment_data].Value))
            if len(self.unemployment_values) > 3:
                self.unemployment_values.pop(0)

        # Credit Spreads
        if self.credit_spread in data and data[self.credit_spread] is not None:
            self.credit_spread_values.append(float(data[self.credit_spread].Value))
            if len(self.credit_spread_values) > 3:
                self.credit_spread_values.pop(0)

        # Consumer Confidence
        if self.confidence_data in data and data[self.confidence_data] is not None:
            self.confidence_values.append(float(data[self.confidence_data].Value))
            if len(self.confidence_values) > 3:
                self.confidence_values.pop(0)

        # **Update Trailing Stop-Loss**
        spy_price = self.Securities[self.spy].Price
        if spy_price > self.spy_trailing_high:
            self.spy_trailing_high = spy_price

        tlt_price = self.Securities[self.long_bonds].Price
        if tlt_price > self.tlt_trailing_high:
            self.tlt_trailing_high = tlt_price

        lqd_price = self.Securities[self.corp_bonds].Price
        if lqd_price > self.lqd_trailing_high:
            self.lqd_trailing_high = lqd_price

    def Rebalance(self):
        if len(self.gdp_values) < 1 or len(self.unemployment_values) < 1 or len(self.credit_spread_values) < 1 or len(self.confidence_values) < 1:
            return  # Ensure data is available before proceeding

        # Get the most recent values
        latest_gdp_growth = self.gdp_values[-1]
        latest_unemployment = self.unemployment_values[-1]
        latest_credit_spread = self.credit_spread_values[-1]
        latest_confidence = self.confidence_values[-1]
        yield_spread = self.Securities[self.yield_10y].Price - self.Securities[self.yield_2y].Price
        spy_price = self.Securities[self.spy].Price
        tlt_price = self.Securities[self.long_bonds].Price
        lqd_price = self.Securities[self.corp_bonds].Price
        current_equity = self.Portfolio.TotalPortfolioValue  

        # **Debugging: Print Current Macro Data**
        self.Debug(f"Date: {self.Time} | GDP: {latest_gdp_growth:.2f} | Unemployment: {latest_unemployment:.2f}% | Yield Spread: {yield_spread:.2f} | Credit Spread: {latest_credit_spread:.2f} | Consumer Confidence: {latest_confidence:.2f}")

        # **Portfolio-Wide Max Drawdown Stop-Loss**
        drawdown_threshold = 0.85  # 15% max drawdown limit
        if current_equity > self.portfolio_peak:
            self.portfolio_peak = current_equity  

        if current_equity < self.portfolio_peak * drawdown_threshold:
            self.Debug(f"Max Drawdown Exceeded! Liquidating Portfolio | Equity: {current_equity:.2f}")
            self.Liquidate()  
            self.SetHoldings(self.tbill, 1)  
            return

        # **ATR-Based Stop-Loss for SPY**
        atr_value = self.atr.Current.Value
        STOP_LOSS_MULTIPLIER = 2  
        if spy_price < self.spy_trailing_high - (STOP_LOSS_MULTIPLIER * atr_value):
            self.Debug(f"ATR Stop-Loss Triggered for SPY at {spy_price:.2f}")
            self.SetHoldings(self.tbill, 0.6)
            self.SetHoldings(self.gold, 0.2)
            self.SetHoldings(self.long_bonds, 0.2)
            return

        # **Bond Momentum Check (Avoid Bonds in Downtrend)**
        if tlt_price < self.tlt_sma.Current.Value:
            self.Debug(f" TLT Below 50-Day SMA, Avoiding Bonds")
            return  

        # **Portfolio Adjustments Based on Economic Conditions**
        if latest_gdp_growth > 2.8 and latest_unemployment < 4.0 and latest_confidence > 100 and yield_spread > 0:
            self.Debug("Bull Market Signal: Increasing SPY Allocation")
            self.SetHoldings(self.spy, 0.7)
            self.SetHoldings(self.gold, 0.15)
            self.SetHoldings(self.corp_bonds, 0.15)

        elif latest_gdp_growth < 2 and latest_unemployment > 4.0 and latest_confidence < 80:
            self.Debug("Recession Signal: Moving to Defensive Assets")
            self.SetHoldings(self.tbill, 0.6)
            self.SetHoldings(self.gold, 0.4)
            self.SetHoldings(self.long_bonds, 0.0)

        else:
            self.Debug("Neutral Market Signal: Balanced Allocation")
            self.SetHoldings(self.spy, 0.5)
            self.SetHoldings(self.gold, 0.25)
            self.SetHoldings(self.corp_bonds, 0.25)
